In [136]:
import os
import pandas as pd
import spacy

from dotenv import load_dotenv
from scipy.sparse import coo_matrix
from IPython.display import display
from gensim import corpora, models, similarities

load_dotenv()

DB_URL = os.getenv('DB_URL')

pd.set_option('display.max_columns', 50)

In [137]:
from pymongo import MongoClient, InsertOne

client = MongoClient(DB_URL)
db = client['alkoholove']

In [122]:
ALCOHOL_COLUMNS = ['_id', 'name', 'kind', 'type', 'description', 'taste', 'aroma', 'finish', 'country']
# name, kind, type, description, taste, aroma, finish, country
alcohols_collection = db['alcohols']
num_topics = len(alcohols_collection.distinct("type"))

items = list(alcohols_collection.find({}, {field_name: 1 for field_name in ALCOHOL_COLUMNS}))
display(items)
items_data = [
    f'{item["name"]}, {item["kind"]}, {item["type"]}, {item["description"]} {", ".join(item["taste"])} {", ".join(item["aroma"])} {", ".join(item["finish"])}, {item["country"]}'
    for item in items
]

[{'_id': ObjectId('62aa2b37e33ccae4961a4daa'),
  'name': 'Jägermeister',
  'kind': 'likier',
  'type': 'Ziołowy',
  'description': 'Skład likieru Jägermeister od niemal stu lat utrzymywana jest w tajemnicy. Jak wiadomo z etykiety, likier zawiera aż 56 ziół i przypraw, które od lat niezmiennie budują smak i aromat trunku.',
  'country': 'Niemcy',
  'finish': [],
  'aroma': ['imbir', 'goździki'],
  'taste': ['imbir', 'skórki cytrusów', 'ziołowy']},
 {'_id': ObjectId('62ab28e88a757f60cc3f31db'),
  'name': ' Biały Bocian Słony Karmel',
  'kind': 'likier',
  'type': 'mleczny',
  'description': 'Mleczny likier o modnym smaku słonego karmelu. Bardzo eleganckie połączenie karmelu, mlecznej, białej czekolady, nugatu, ale wyczuwalna jest już w zapachu słoność. Przyjemny smak, delikatny, nie za słodki – biała czekolada, orzechy, finisz to harmonijne połączenie sporej porcji słodyczy ze słonością.',
  'country': 'Polska',
  'finish': ['słodki', 'słony'],
  'aroma': ['słony karmel', 'biała czekolad

In [123]:
documents = []
num_topics

244

In [124]:
nlp = spacy.load("pl_core_news_lg")
docs = nlp.pipe(items_data)

In [125]:
for doc in docs:
    documents.append([token.lemma_ for token in doc if (not token.is_stop and not token.is_punct)])

In [126]:
documents

[['jägermeister',
  'likier',
  'Ziołowy',
  'skład',
  'likier',
  'jägermeister',
  'niemal',
  'sto',
  'rok',
  'utrzymywać',
  'tajemnica',
  'wiadomo',
  'etykieta',
  'likier',
  'zawierać',
  '56',
  'zioło',
  'przyprawa',
  'rok',
  'niezmiennie',
  'budować',
  'smak',
  'aromat',
  'trunek',
  'imbir',
  'skórka',
  'cytrus',
  'ziołowy',
  'imbir',
  'goździk',
  'niemiec'],
 [' ',
  'biały',
  'bocian',
  'Słony',
  'Karmel',
  'likier',
  'mleczny',
  'mleczny',
  'likier',
  'modny',
  'smak',
  'słony',
  'karmel',
  'elegancki',
  'połączyć',
  'karmel',
  'mleczny',
  'biały',
  'czekolada',
  'nugat',
  'wyczuwalny',
  'zapach',
  'słoność',
  'przyjemny',
  'smak',
  'delikatny',
  'słodki',
  'biały',
  'czekolada',
  'orzech',
  'finisz',
  'harmonijny',
  'połączyć',
  'spory',
  'porcja',
  'słodycz',
  'słoność',
  'biały',
  'czekolada',
  'Wanillia',
  'delikatny',
  'słony',
  'karmel',
  'biały',
  'czekolada',
  'nugat',
  'morski',
  'sól',
  'słodki',
 

In [127]:
dictionary = corpora.Dictionary(documents)
corpus = [dictionary.doc2bow(document) for document in documents]
lda_model = models.ldamodel.LdaModel(
    corpus=corpus,
    id2word=dictionary,
    num_topics=num_topics)
index = similarities.MatrixSimilarity(corpus)

In [128]:
# save everything above?

In [129]:
coo = coo_matrix(index)
csr = coo.tocsr()

In [130]:
xs, ys = coo.nonzero()

operations = []
for x, y in zip(xs, ys):

    if x == y:
        continue

    sim = float(csr[x, y]) #similarity
    if sim < 0.1:
        continue
    x_id = str(items[x]['_id']) #source
    y_id = str(items[y]['_id']) #target
    operations.append(InsertOne({'source': x_id, 'target': y_id, 'sim': sim}))
    # save to db

In [131]:
db['lda_sim'].bulk_write(operations)

In [ ]:
# zapisane w bazie, zaimplementować by serwować:
# https://github.dev/practical-recommender-systems/moviegeek/blob/ca994bad8bad4a662e8dd65a658e0e4d16d427e8/recs/content_based_recommender.py

In [135]:
lda_model.print_topics()

[(85,
  '0.031*"chmiel" + 0.028*"wiśniowy" + 0.021*"Polska" + 0.021*"Lubelska" + 0.020*"goryczka" + 0.019*"piwo" + 0.019*"Zwierzyniecki" + 0.019*"likier" + 0.019*"zapach" + 0.018*"wiśniówka"'),
 (103,
  '0.028*"piwo" + 0.025*"grejpfrut" + 0.024*"cytrus" + 0.024*"cytrusowy" + 0.024*"chmiel" + 0.024*"Witbier" + 0.020*"Polska" + 0.020*"orzeźwiający" + 0.019*"dodatek" + 0.015*"absolwent"'),
 (230,
  '0.037*"Polska" + 0.036*"piwo" + 0.025*"smak" + 0.022*"piwny" + 0.021*"gruszka" + 0.021*"napój" + 0.017*"  " + 0.016*"czysty" + 0.014*"chmielowy" + 0.012*"słodowy"'),
 (232,
  '0.026*"piwo" + 0.024*"Polska" + 0.020*"smak" + 0.017*"słód" + 0.016*"dubeltowy" + 0.016*"chmiel" + 0.013*"słodki" + 0.013*"jasny" + 0.012*"goryczka" + 0.011*"pełny"'),
 (46,
  '0.046*"piwo" + 0.033*"Brain" + 0.033*"Moczybroda" + 0.023*"owocowy" + 0.022*"Polska" + 0.020*"goryczka" + 0.017*"typ" + 0.017*"ipa" + 0.017*"chmielć" + 0.017*"India"'),
 (199,
  '0.036*"wanilia" + 0.022*"rum" + 0.022*"czekolada" + 0.018*"dębina" +